# Project - Third Requirement

### Include packages.

In [1]:
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma
from itertools import product
from scipy.optimize import linprog

### As usual, as first, define the environment.

In [2]:
class Environment:
    def __init__(self):
        pass
    def round(self, a_t):
        pass

In [ ]:
#In this case the valuation of the product is sampled from different distribution at each round
# dist_1_params and dist_2_params contains the parameters of the two distribution from which the valuations will be sampled
class PricingEnvironmentSingleProductMultiDistribution(Environment):
    def __init__(self, cost, dist_1_params, dist_2_params, buyers_per_round, std_val):
        self.N = len(cost)
        self.cost = cost
        self.dist_1_params = dist_1_params
        self.dist_2_params = dist_2_params
        self.buyers_per_round = buyers_per_round
        self.std_val = std_val

    def round(self, p_t, dist):
        # Simply if dist is even sample from dist_1, if dist is odd sample from dist_2
        # In this case the valuation of the buyers change at each round
        if dist % 2 == 0:
            mean_val = self.dist_1_params[0]
            std_val = self.dist_1_params[1]
            sample_valuation = norm.rvs(loc=mean_val, scale=std_val)
        else:
            shape = self.dist_2_params[0]
            scale = self.dist_2_params[1]
            sample_valuation = gamma.rvs(a=shape, scale=scale)
        
        # Use as mean of the evaluation the sample from the previous distributions
        # Probability of a single customer buying at price p_t
        prob_buy = 1 - norm.cdf(p_t, loc=sample_valuation, scale=self.std_val)
        
        # Number of sales is drawn from a binomial distribution
        num_of_sales = np.random.binomial(n=self.buyers_per_round, p=prob_buy)
        
        reward = (p_t - self.cost) * num_of_sales
        return num_of_sales, reward
            
